# Generating TOA Radiances with OSOAA for Atmospheric Correction

This notebook uses the OSOAA (Ocean Successive Orders with Atmosphere - Advanced) radiative transfer code to generate simulated top-of-atmosphere (TOA) radiance values. These synthetic observations can then be used to test and validate the `correct_atmosphere` package's ability to recover water-leaving radiances.

## Overview

OSOAA is a vector radiative transfer model for coupled atmosphere-ocean systems that:
- Computes full Stokes vector (I, Q, U, V) radiances
- Uses successive orders of scattering method
- Includes Rayleigh scattering, aerosols, ocean surface, and hydrosols

### Workflow

1. Configure OSOAA with realistic atmospheric and oceanic parameters
2. Run simulations at ocean color wavelengths (matching MODIS/VIIRS bands)
3. Extract TOA radiances at satellite viewing geometries
4. (Optionally) Apply the `correct_atmosphere` algorithm to recover water-leaving radiances
5. Compare recovered Lw with OSOAA's "true" water-leaving radiance

## 1. Setup and Imports

In [ ]:
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import os
import tempfile
import shutil
from pathlib import Path
from typing import Dict, Optional, List, Tuple
from dataclasses import dataclass

# correct_atmosphere imports
import correct_atmosphere as ca
from correct_atmosphere import constants
from correct_atmosphere.correction import (
    AtmosphericCorrection,
    GeometryAngles,
    AncillaryData,
)

# Configure OSOAA paths
# Adjust this path to your OSOAA installation
OSOAA_ROOT = Path(os.environ.get(
    'OSOAA_ROOT', 
    '/home/xavier/Projects/Oceanography/python/RadiativeTransferCode-OSOAA'
)).resolve()

OSOAA_EXE = OSOAA_ROOT / 'exe' / 'OSOAA_MAIN.exe'

print(f"OSOAA Root: {OSOAA_ROOT}")
print(f"Executable exists: {OSOAA_EXE.exists()}")

## 2. OSOAA Simulation Class

This class wraps the OSOAA Fortran executable and provides methods to:
- Configure simulations with appropriate parameters
- Run both TOA and subsurface simulations
- Parse output files into Python dictionaries

**Important OSOAA conventions:**
- Wavelength is in **micrometers** (not nanometers)
- Imaginary refractive indices are **negative** (absorption convention)
- Output radiances are normalized: $\pi L / E_{sun}$

In [ ]:
class OSOAASimulation:
    """
    Python wrapper for OSOAA radiative transfer simulations.
    
    Supports both TOA and subsurface radiance computations.
    """
    
    def __init__(self, osoaa_root: Path, work_dir: Optional[Path] = None):
        self.osoaa_root = Path(osoaa_root).resolve()
        self.exe_path = self.osoaa_root / 'exe' / 'OSOAA_MAIN.exe'
        self.fic_path = self.osoaa_root / 'fic'
        
        # Create working directory with required subdirectories
        if work_dir:
            self.work_dir = Path(work_dir).resolve()
        else:
            self.work_dir = Path(tempfile.mkdtemp(prefix='osoaa_toa_'))
        
        # Create directories for Mie calculations and surface matrices
        self.mie_aer_dir = self.work_dir / 'MIE_AER'
        self.mie_hyd_dir = self.work_dir / 'MIE_HYD'
        self.surf_dir = self.work_dir / 'SURF'
        
        self.mie_aer_dir.mkdir(parents=True, exist_ok=True)
        self.mie_hyd_dir.mkdir(parents=True, exist_ok=True)
        self.surf_dir.mkdir(parents=True, exist_ok=True)
        
        if not self.exe_path.exists():
            raise FileNotFoundError(f"OSOAA executable not found at {self.exe_path}")
        
        print(f"Working directory: {self.work_dir}")
    
    def get_toa_params(
        self,
        wavelength_nm: float = 550.0,
        solar_zenith: float = 30.0,
        view_zenith: float = 0.0,
        relative_azimuth: float = 90.0,
        chlorophyll: float = 0.1,
        aot: float = 0.1,
        wind_speed: float = 5.0,
        pressure: float = 1013.25,
        ozone_du: float = 300.0,
    ) -> Dict:
        """
        Generate parameters for TOA radiance simulation.
        
        Parameters
        ----------
        wavelength_nm : float
            Wavelength in nanometers
        solar_zenith : float
            Solar zenith angle in degrees (0-90)
        view_zenith : float
            Viewing zenith angle in degrees (positive = toward sun)
        relative_azimuth : float
            Relative azimuth angle in degrees (0 = forward scatter, 180 = backscatter)
        chlorophyll : float
            Chlorophyll-a concentration in mg/m^3
        aot : float
            Aerosol optical thickness at the wavelength
        wind_speed : float
            Wind speed in m/s (for Cox-Munk sea surface)
        pressure : float
            Surface pressure in hPa
        ozone_du : float
            Ozone column in Dobson Units
        
        Returns
        -------
        dict
            OSOAA parameter dictionary
        """
        wavelength_um = wavelength_nm / 1000.0
        
        # Convert ozone from Dobson Units to atm-cm (1 DU = 0.001 atm-cm)
        ozone_atm_cm = ozone_du * 0.001
        
        return {
            # Working directory
            'OSOAA.ResRoot': str(self.work_dir),
            
            # Wavelength in micrometers
            'OSOAA.Wa': wavelength_um,
            
            # Solar geometry
            'ANG.Thetas': solar_zenith,
            
            # Viewing geometry
            'OSOAA.View.Phi': relative_azimuth,
            'OSOAA.View.Level': 1,  # 1 = Top of Atmosphere
            'OSOAA.View.Z': 0.0,
            
            # Atmospheric profile
            'AP.Pressure': pressure,
            'AP.HR': 8.0,      # Rayleigh scale height (km)
            'AP.HA': 2.0,      # Aerosol scale height (km)
            'AP.MOT': ozone_atm_cm,  # Ozone content (atm-cm)
            
            # Aerosol model - maritime-type aerosol
            'AER.DirMie': str(self.mie_aer_dir),
            'AER.Waref': wavelength_um,
            'AER.AOTref': aot,
            'AER.Model': 0,            # Mono-modal
            'AER.MMD.MRwa': 1.40,      # Real refractive index (maritime)
            'AER.MMD.MIwa': -0.001,    # Imaginary refractive index
            'AER.MMD.SDtype': 1,       # Log-normal distribution
            'AER.MMD.LNDradius': 0.10, # Modal radius (um)
            'AER.MMD.LNDvar': 0.46,    # Log of standard deviation
            
            # Sea profile
            'SEA.Depth': 1000.0,       # Deep ocean
            
            # Hydrosol model - phytoplankton
            'HYD.DirMie': str(self.mie_hyd_dir),
            'HYD.Model': 1,
            'PHYTO.Chl': chlorophyll,
            'PHYTO.ProfilType': 1,     # Homogeneous profile
            
            # Phytoplankton optical properties
            'PHYTO.JD.slope': 4.0,
            'PHYTO.JD.rmin': 0.01,
            'PHYTO.JD.rmax': 200.0,
            'PHYTO.JD.MRwa': 1.05,
            'PHYTO.JD.MIwa': 0.0,
            'PHYTO.JD.rate': 1.0,
            
            # Other ocean constituents
            'SED.Csed': 0.0,
            'YS.Abs440': 0.0,
            'DET.Abs440': 0.0,
            
            # Sea surface (Cox-Munk)
            'SEA.Dir': str(self.surf_dir),
            'SEA.Ind': 1.34,
            'SEA.Wind': wind_speed,
            'SEA.SurfAlb': 0.0,
            'SEA.BotType': 1,
            'SEA.BotAlb': 0.0,         # Deep ocean, no bottom contribution
            
            # Output files
            'OSOAA.ResFile.vsVZA': 'LUM_vsVZA.txt',
        }
    
    def get_subsurface_params(
        self,
        wavelength_nm: float = 550.0,
        solar_zenith: float = 30.0,
        chlorophyll: float = 0.1,
        wind_speed: float = 5.0,
    ) -> Dict:
        """
        Generate parameters for water-leaving radiance (just below surface).
        
        This gives the "true" Lw that atmospheric correction should recover.
        """
        wavelength_um = wavelength_nm / 1000.0
        
        return {
            'OSOAA.ResRoot': str(self.work_dir),
            'OSOAA.Wa': wavelength_um,
            'ANG.Thetas': solar_zenith,
            'OSOAA.View.Phi': 90.0,
            'OSOAA.View.Level': 4,   # 4 = Just below sea surface (0-)
            'OSOAA.View.Z': 0.0,
            
            # Minimal atmosphere for water-leaving calculation
            'AP.Pressure': 1013.0,
            'AP.HR': 8.0,
            'AP.HA': 2.0,
            
            # Very low aerosol for clean atmosphere
            'AER.DirMie': str(self.mie_aer_dir),
            'AER.Waref': wavelength_um,
            'AER.AOTref': 0.01,  # Nearly aerosol-free
            'AER.Model': 0,
            'AER.MMD.MRwa': 1.40,
            'AER.MMD.MIwa': -0.001,
            'AER.MMD.SDtype': 1,
            'AER.MMD.LNDradius': 0.10,
            'AER.MMD.LNDvar': 0.46,
            
            # Ocean
            'SEA.Depth': 1000.0,
            'HYD.DirMie': str(self.mie_hyd_dir),
            'HYD.Model': 1,
            'PHYTO.Chl': chlorophyll,
            'PHYTO.ProfilType': 1,
            'PHYTO.JD.slope': 4.0,
            'PHYTO.JD.rmin': 0.01,
            'PHYTO.JD.rmax': 200.0,
            'PHYTO.JD.MRwa': 1.05,
            'PHYTO.JD.MIwa': 0.0,
            'PHYTO.JD.rate': 1.0,
            
            'SED.Csed': 0.0,
            'YS.Abs440': 0.0,
            'DET.Abs440': 0.0,
            
            'SEA.Dir': str(self.surf_dir),
            'SEA.Ind': 1.34,
            'SEA.Wind': wind_speed,
            'SEA.SurfAlb': 0.0,
            'SEA.BotType': 1,
            'SEA.BotAlb': 0.0,
            
            'OSOAA.ResFile.vsVZA': 'LUM_vsVZA_Lw.txt',
        }
    
    def build_command(self, params: Dict) -> List[str]:
        """Build command-line arguments for OSOAA."""
        cmd = [str(self.exe_path)]
        for key, value in params.items():
            cmd.extend([f'-{key}', str(value)])
        return cmd
    
    def run(self, params: Dict, verbose: bool = True, timeout: int = 600) -> Dict:
        """Run OSOAA simulation and return parsed results."""
        cmd = self.build_command(params)
        
        if verbose:
            wl_nm = params.get('OSOAA.Wa', 0) * 1000
            level = params.get('OSOAA.View.Level', 1)
            level_name = {1: 'TOA', 4: 'Subsurface'}.get(level, f'Level {level}')
            print(f"Running OSOAA ({level_name}) at {wl_nm:.0f} nm...")
        
        result = subprocess.run(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            cwd=str(self.osoaa_root),
            env={**os.environ, 'OSOAA_ROOT': str(self.osoaa_root)},
            timeout=timeout
        )
        
        if result.returncode != 0:
            print(f"OSOAA Error (return code {result.returncode}):")
            print("STDERR:", result.stderr[:2000] if result.stderr else "(empty)")
            raise RuntimeError("OSOAA simulation failed")
        
        if verbose:
            print("  Done.")
        
        return self.parse_results(params)
    
    def parse_results(self, params: Dict) -> Dict:
        """Parse OSOAA output files."""
        results = {'params': params}
        output_dir = self.work_dir / 'Standard_outputs'
        
        vza_filename = params.get('OSOAA.ResFile.vsVZA', 'LUM_vsVZA.txt')
        vza_file = output_dir / vza_filename
        
        if vza_file.exists():
            results['vza_data'] = self._parse_vza_file(vza_file)
            results['output_file'] = str(vza_file)
        else:
            print(f"Warning: Output file not found: {vza_file}")
        
        return results
    
    def _parse_vza_file(self, filepath: Path) -> Dict:
        """Parse LUM_vsVZA.txt output."""
        with open(filepath, 'r') as f:
            lines = f.readlines()
        
        data_start = 0
        for i, line in enumerate(lines):
            if line.strip().startswith('VZA') and 'SCA_ANG' in line:
                data_start = i + 1
                break
        
        data = np.loadtxt(filepath, skiprows=data_start)
        
        return {
            'vza': data[:, 0],              # Viewing Zenith Angle (degrees)
            'scattering_angle': data[:, 1], # Scattering angle (degrees)
            'I': data[:, 2],                # Normalized radiance (pi*L/Esun)
            'reflectance': data[:, 3],      # Reflectance (pi*L/Ed)
            'DoLP': data[:, 4],             # Degree of polarization (%)
            'I_pol': data[:, 5],            # Polarized intensity
            'refl_pol': data[:, 6],         # Polarized reflectance
        }
    
    def get_radiance_at_vza(self, results: Dict, target_vza: float) -> float:
        """Extract radiance at a specific viewing zenith angle."""
        if 'vza_data' not in results:
            return np.nan
        vza = results['vza_data']['vza']
        I = results['vza_data']['I']
        idx = np.argmin(np.abs(vza - target_vza))
        return I[idx]
    
    def cleanup(self):
        """Remove working directory."""
        if self.work_dir.exists():
            shutil.rmtree(self.work_dir, ignore_errors=True)


print("OSOAASimulation class defined.")

## 3. Define Simulation Scenario

We'll simulate a realistic ocean color observation scenario:

| Parameter | Value | Description |
|-----------|-------|-------------|
| Solar Zenith | 30° | Mid-morning or afternoon sun |
| View Zenith | 15° | Off-nadir satellite viewing |
| Relative Azimuth | 90° | Cross-principal plane |
| Chlorophyll | 0.3 mg/m³ | Moderate open ocean |
| AOT (550) | 0.1 | Typical maritime aerosol |
| Wind Speed | 5 m/s | Moderate sea state |
| Ozone | 300 DU | Standard ozone |

### Wavelengths

We'll use common ocean color bands that match MODIS-Aqua:

In [ ]:
@dataclass
class SimulationScenario:
    """Container for simulation parameters."""
    name: str
    solar_zenith: float
    view_zenith: float
    relative_azimuth: float
    chlorophyll: float
    aot_550: float
    wind_speed: float
    pressure: float = 1013.25
    ozone: float = 300.0
    
    def __repr__(self):
        return (f"{self.name}: SZA={self.solar_zenith}°, VZA={self.view_zenith}°, "
                f"RAA={self.relative_azimuth}°, Chl={self.chlorophyll} mg/m³, "
                f"AOT={self.aot_550}")


# Define our test scenario
scenario = SimulationScenario(
    name="Open Ocean - Moderate",
    solar_zenith=30.0,
    view_zenith=15.0,
    relative_azimuth=90.0,
    chlorophyll=0.3,
    aot_550=0.10,
    wind_speed=5.0,
    pressure=1013.25,
    ozone=300.0
)

print("Simulation scenario:")
print(scenario)

In [ ]:
# Ocean color wavelengths (matching MODIS-Aqua visible bands)
WAVELENGTHS_NM = [
    412,   # Blue (deep chlorophyll absorption)
    443,   # Blue (chlorophyll absorption)
    488,   # Blue-green
    531,   # Green
    547,   # Green
    667,   # Red (chlorophyll fluorescence)
    678,   # Red
    748,   # NIR (aerosol correction)
    869,   # NIR (aerosol correction)
]

print(f"Wavelengths to simulate: {WAVELENGTHS_NM} nm")
print(f"Number of bands: {len(WAVELENGTHS_NM)}")

## 4. Run TOA Radiance Simulations

We'll run OSOAA at each wavelength and collect TOA radiances. The AOT is scaled spectrally using an Angstrom exponent.

In [ ]:
def scale_aot_angstrom(aot_ref: float, wl_ref: float, wl_target: float, 
                        angstrom: float = 1.0) -> float:
    """
    Scale AOT using Angstrom exponent.
    
    AOT(λ) = AOT(λ_ref) * (λ/λ_ref)^(-α)
    
    Parameters
    ----------
    aot_ref : float
        AOT at reference wavelength
    wl_ref : float
        Reference wavelength (nm)
    wl_target : float
        Target wavelength (nm)
    angstrom : float
        Angstrom exponent (typically 0.5-2.0 for maritime aerosols)
    
    Returns
    -------
    float
        AOT at target wavelength
    """
    return aot_ref * (wl_target / wl_ref) ** (-angstrom)


# Test AOT scaling
angstrom_exp = 1.0  # Typical for maritime aerosol
print("AOT at each wavelength (Angstrom exponent = 1.0):")
print("-" * 40)
for wl in WAVELENGTHS_NM:
    aot = scale_aot_angstrom(scenario.aot_550, 550.0, wl, angstrom_exp)
    print(f"  {wl:3d} nm: AOT = {aot:.4f}")

In [ ]:
# Initialize simulation
sim = OSOAASimulation(OSOAA_ROOT)

# Run TOA simulations at all wavelengths
toa_results = {}
angstrom_exp = 1.0

print("Running TOA radiance simulations...")
print("=" * 50)

for wl in WAVELENGTHS_NM:
    # Scale AOT for this wavelength
    aot_wl = scale_aot_angstrom(scenario.aot_550, 550.0, wl, angstrom_exp)
    
    # Get TOA parameters
    params = sim.get_toa_params(
        wavelength_nm=float(wl),
        solar_zenith=scenario.solar_zenith,
        view_zenith=scenario.view_zenith,
        relative_azimuth=scenario.relative_azimuth,
        chlorophyll=scenario.chlorophyll,
        aot=aot_wl,
        wind_speed=scenario.wind_speed,
        pressure=scenario.pressure,
        ozone_du=scenario.ozone,
    )
    params['OSOAA.ResFile.vsVZA'] = f'LUM_TOA_{wl}nm.txt'
    
    try:
        results = sim.run(params, verbose=True)
        toa_results[wl] = results
    except Exception as e:
        print(f"  ERROR at {wl} nm: {e}")

print("\nTOA simulations completed!")
print(f"Successful: {len(toa_results)} / {len(WAVELENGTHS_NM)} wavelengths")

## 5. Extract TOA Radiances at Satellite Viewing Geometry

OSOAA outputs radiance as a function of viewing zenith angle. We need to extract the value at our specified satellite viewing geometry.

In [ ]:
# Extract TOA radiances at the specified viewing angle
target_vza = scenario.view_zenith

toa_radiances = {}  # Normalized radiance (pi*L/Esun)

print(f"TOA Radiances at VZA = {target_vza}°:")
print("=" * 50)
print(f"{'Wavelength (nm)':<18} {'TOA Radiance':>15} {'AOT':>10}")
print("-" * 50)

for wl in sorted(toa_results.keys()):
    results = toa_results[wl]
    rho_toa = sim.get_radiance_at_vza(results, target_vza)
    toa_radiances[wl] = rho_toa
    
    aot_wl = scale_aot_angstrom(scenario.aot_550, 550.0, wl, angstrom_exp)
    print(f"{wl:>8} nm        {rho_toa:>15.6f} {aot_wl:>10.4f}")

print("-" * 50)

## 6. Simulate "True" Water-Leaving Radiance

To validate atmospheric correction, we also need the "true" water-leaving radiance that should be recovered. We run OSOAA at the subsurface level (just below the water surface) to get this reference.

In [ ]:
# Run water-leaving (subsurface) simulations
lw_results = {}

print("Running water-leaving radiance simulations...")
print("=" * 50)

for wl in WAVELENGTHS_NM:
    params = sim.get_subsurface_params(
        wavelength_nm=float(wl),
        solar_zenith=scenario.solar_zenith,
        chlorophyll=scenario.chlorophyll,
        wind_speed=scenario.wind_speed,
    )
    params['OSOAA.ResFile.vsVZA'] = f'LUM_Lw_{wl}nm.txt'
    
    try:
        results = sim.run(params, verbose=True)
        lw_results[wl] = results
    except Exception as e:
        print(f"  ERROR at {wl} nm: {e}")

print("\nWater-leaving simulations completed!")

In [ ]:
# Extract water-leaving radiances
lw_radiances = {}

print(f"Water-Leaving Radiances (subsurface at nadir):")
print("=" * 50)
print(f"{'Wavelength (nm)':<18} {'Lw (normalized)':>18}")
print("-" * 50)

for wl in sorted(lw_results.keys()):
    results = lw_results[wl]
    # For water-leaving, we typically want nadir
    rho_lw = sim.get_radiance_at_vza(results, 0.0)
    lw_radiances[wl] = rho_lw
    print(f"{wl:>8} nm        {rho_lw:>18.6f}")

print("-" * 50)

## 7. Visualize Simulated Spectra

Let's visualize the TOA radiance spectrum and compare it with the water-leaving radiance.

In [ ]:
def plot_spectra(toa_radiances: Dict, lw_radiances: Dict, scenario: SimulationScenario):
    """
    Plot TOA and water-leaving radiance spectra.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Common wavelengths
    wavelengths = sorted(set(toa_radiances.keys()) & set(lw_radiances.keys()))
    
    toa_vals = [toa_radiances[wl] for wl in wavelengths]
    lw_vals = [lw_radiances[wl] for wl in wavelengths]
    
    # Left panel: Both spectra
    ax = axes[0]
    ax.plot(wavelengths, toa_vals, 'ko-', markersize=10, linewidth=2, 
            label='TOA Radiance')
    ax.plot(wavelengths, lw_vals, 'bs--', markersize=8, linewidth=2, 
            label='Water-Leaving Radiance')
    ax.set_xlabel('Wavelength (nm)', fontsize=12)
    ax.set_ylabel(r'Normalized Radiance ($\pi L / E_{sun}$)', fontsize=12)
    ax.set_title('TOA vs Water-Leaving Radiance', fontsize=14)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_xlim(380, 900)
    
    # Right panel: Atmospheric contribution
    ax = axes[1]
    atm_contribution = [toa - lw for toa, lw in zip(toa_vals, lw_vals)]
    atm_fraction = [100 * (toa - lw) / toa if toa > 0 else 0 
                    for toa, lw in zip(toa_vals, lw_vals)]
    
    ax.bar(wavelengths, atm_fraction, width=15, color='orange', alpha=0.7, 
           edgecolor='black')
    ax.set_xlabel('Wavelength (nm)', fontsize=12)
    ax.set_ylabel('Atmospheric Contribution (%)', fontsize=12)
    ax.set_title('Fraction of TOA Signal from Atmosphere', fontsize=14)
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_xlim(380, 900)
    ax.set_ylim(0, 100)
    
    # Add reference line at 80%
    ax.axhline(y=80, color='red', linestyle='--', linewidth=1.5, 
               label='Typical ocean color (~80-90%)')
    ax.legend(fontsize=10)
    
    fig.suptitle(f'Scenario: {scenario.name}\n'
                 f'SZA={scenario.solar_zenith}°, Chl={scenario.chlorophyll} mg/m³, '
                 f'AOT(550)={scenario.aot_550}',
                 fontsize=12, y=1.02)
    
    plt.tight_layout()
    return fig


if toa_radiances and lw_radiances:
    plot_spectra(toa_radiances, lw_radiances, scenario)
    plt.show()

## 8. Angular Distribution of TOA Radiance

Let's also look at how TOA radiance varies with viewing angle at a selected wavelength.

In [ ]:
def plot_angular_distribution(toa_results: Dict, wavelength: int = 550):
    """
    Plot TOA radiance angular distribution at a given wavelength.
    """
    # Find closest wavelength in results
    available_wls = list(toa_results.keys())
    closest_wl = min(available_wls, key=lambda x: abs(x - wavelength))
    
    results = toa_results[closest_wl]
    if 'vza_data' not in results:
        print(f"No angular data available for {closest_wl} nm")
        return
    
    vza = results['vza_data']['vza']
    radiance = results['vza_data']['I']
    dolp = results['vza_data']['DoLP']
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Radiance vs VZA
    ax = axes[0]
    ax.plot(vza, radiance, 'b-', linewidth=2)
    ax.axvline(x=scenario.view_zenith, color='red', linestyle='--', 
               linewidth=1.5, label=f'Satellite VZA = {scenario.view_zenith}°')
    ax.set_xlabel('Viewing Zenith Angle (degrees)', fontsize=12)
    ax.set_ylabel(r'TOA Radiance ($\pi L / E_{sun}$)', fontsize=12)
    ax.set_title(f'TOA Radiance vs Viewing Angle ({closest_wl} nm)', fontsize=14)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Polarization vs VZA
    ax = axes[1]
    ax.plot(vza, dolp, 'm-', linewidth=2)
    ax.axvline(x=scenario.view_zenith, color='red', linestyle='--', 
               linewidth=1.5, label=f'Satellite VZA = {scenario.view_zenith}°')
    ax.set_xlabel('Viewing Zenith Angle (degrees)', fontsize=12)
    ax.set_ylabel('Degree of Linear Polarization (%)', fontsize=12)
    ax.set_title(f'Polarization vs Viewing Angle ({closest_wl} nm)', fontsize=14)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


if toa_results:
    # Plot for a wavelength close to 550 nm
    plot_angular_distribution(toa_results, wavelength=547)
    plt.show()

## 9. Create Dataset for Atmospheric Correction

Now let's package the OSOAA outputs into a format suitable for the `correct_atmosphere` package.

In [ ]:
@dataclass
class OSOAADataset:
    """
    Container for OSOAA simulation results formatted for atmospheric correction.
    
    Attributes
    ----------
    wavelengths : ndarray
        Wavelengths in nm
    toa_reflectance : ndarray
        TOA reflectance (pi*Lt/F0) at each wavelength
    true_rho_w : ndarray
        "True" water-leaving reflectance from OSOAA
    geometry : GeometryAngles
        Observation geometry
    ancillary : AncillaryData
        Ancillary data used in simulation
    scenario : SimulationScenario
        Full scenario parameters
    """
    wavelengths: np.ndarray
    toa_reflectance: np.ndarray
    true_rho_w: np.ndarray
    geometry: GeometryAngles
    ancillary: AncillaryData
    scenario: SimulationScenario
    
    def summary(self):
        """Print summary of dataset."""
        print("OSOAA Simulation Dataset")
        print("=" * 50)
        print(f"Scenario: {self.scenario.name}")
        print(f"Wavelengths: {self.wavelengths} nm")
        print(f"\nGeometry:")
        print(f"  Solar Zenith: {self.geometry.solar_zenith}°")
        print(f"  View Zenith: {self.geometry.view_zenith}°")
        print(f"  Relative Azimuth: {self.geometry.relative_azimuth}°")
        print(f"\nOcean:")
        print(f"  Chlorophyll: {self.scenario.chlorophyll} mg/m³")
        print(f"\nAtmosphere:")
        print(f"  AOT (550 nm): {self.scenario.aot_550}")
        print(f"  Wind Speed: {self.ancillary.wind_speed} m/s")
        print(f"  Ozone: {self.ancillary.ozone} DU")


def create_dataset(toa_radiances: Dict, lw_radiances: Dict, 
                   scenario: SimulationScenario) -> OSOAADataset:
    """
    Create a dataset from OSOAA simulation results.
    """
    # Get common wavelengths
    wavelengths = sorted(set(toa_radiances.keys()) & set(lw_radiances.keys()))
    
    # Extract values
    toa_vals = np.array([toa_radiances[wl] for wl in wavelengths])
    lw_vals = np.array([lw_radiances[wl] for wl in wavelengths])
    
    # Create geometry object
    geometry = GeometryAngles(
        solar_zenith=scenario.solar_zenith,
        solar_azimuth=0.0,  # Reference direction
        view_zenith=scenario.view_zenith,
        view_azimuth=scenario.relative_azimuth,
    )
    
    # Create ancillary data object
    ancillary = AncillaryData(
        pressure=scenario.pressure,
        wind_speed=scenario.wind_speed,
        ozone=scenario.ozone,
        water_vapor=1.5,  # Default
        relative_humidity=80.0,
    )
    
    return OSOAADataset(
        wavelengths=np.array(wavelengths),
        toa_reflectance=toa_vals,
        true_rho_w=lw_vals,
        geometry=geometry,
        ancillary=ancillary,
        scenario=scenario,
    )


# Create dataset
if toa_radiances and lw_radiances:
    dataset = create_dataset(toa_radiances, lw_radiances, scenario)
    dataset.summary()

## 10. Save Dataset for Later Use

Let's save the simulation results so they can be loaded later without re-running OSOAA.

In [ ]:
import json

def save_dataset(dataset: OSOAADataset, filepath: Path):
    """
    Save OSOAA dataset to JSON file.
    """
    data = {
        'wavelengths_nm': dataset.wavelengths.tolist(),
        'toa_reflectance': dataset.toa_reflectance.tolist(),
        'true_rho_w': dataset.true_rho_w.tolist(),
        'geometry': {
            'solar_zenith': float(dataset.geometry.solar_zenith),
            'solar_azimuth': float(dataset.geometry.solar_azimuth),
            'view_zenith': float(dataset.geometry.view_zenith),
            'view_azimuth': float(dataset.geometry.view_azimuth),
        },
        'ancillary': {
            'pressure': float(dataset.ancillary.pressure),
            'wind_speed': float(dataset.ancillary.wind_speed),
            'ozone': float(dataset.ancillary.ozone),
            'water_vapor': float(dataset.ancillary.water_vapor),
            'relative_humidity': float(dataset.ancillary.relative_humidity),
        },
        'scenario': {
            'name': dataset.scenario.name,
            'chlorophyll': dataset.scenario.chlorophyll,
            'aot_550': dataset.scenario.aot_550,
        }
    }
    
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"Dataset saved to: {filepath}")


def load_dataset(filepath: Path) -> Dict:
    """
    Load OSOAA dataset from JSON file.
    """
    with open(filepath, 'r') as f:
        data = json.load(f)
    
    # Convert lists back to numpy arrays
    data['wavelengths_nm'] = np.array(data['wavelengths_nm'])
    data['toa_reflectance'] = np.array(data['toa_reflectance'])
    data['true_rho_w'] = np.array(data['true_rho_w'])
    
    return data


# Save the dataset
if 'dataset' in dir():
    output_path = Path('.') / 'osoaa_toa_dataset.json'
    save_dataset(dataset, output_path)

## 11. Summary: Output Data Dictionary

Here's a summary of what OSOAA provides for atmospheric correction testing:

In [ ]:
if 'dataset' in dir():
    print("OSOAA Output Summary")
    print("=" * 70)
    print(f"\n{'Wavelength (nm)':<15} {'TOA Refl':>12} {'True Lw Refl':>14} {'Atm Frac (%)':>14}")
    print("-" * 70)
    
    for i, wl in enumerate(dataset.wavelengths):
        toa = dataset.toa_reflectance[i]
        lw = dataset.true_rho_w[i]
        atm_frac = 100 * (toa - lw) / toa if toa > 0 else 0
        print(f"{wl:>8.0f}        {toa:>12.6f} {lw:>14.6f} {atm_frac:>14.1f}")
    
    print("-" * 70)
    print("\nNote: All reflectances are normalized (pi*L/E_sun)")
    print("\nThis data can now be used to test atmospheric correction algorithms.")
    print("The goal is to recover 'True Lw Refl' from 'TOA Refl'.")

## 12. Multiple Scenarios (Optional)

Generate TOA data for multiple scenarios to build a more comprehensive test dataset.

In [ ]:
# Define additional test scenarios
test_scenarios = [
    SimulationScenario(
        name="Clear Water - Low Aerosol",
        solar_zenith=30.0,
        view_zenith=15.0,
        relative_azimuth=90.0,
        chlorophyll=0.05,
        aot_550=0.05,
        wind_speed=3.0,
    ),
    SimulationScenario(
        name="Moderate Chl - Moderate Aerosol",
        solar_zenith=45.0,
        view_zenith=20.0,
        relative_azimuth=135.0,
        chlorophyll=0.5,
        aot_550=0.15,
        wind_speed=7.0,
    ),
    SimulationScenario(
        name="High Chl - High Aerosol",
        solar_zenith=50.0,
        view_zenith=30.0,
        relative_azimuth=45.0,
        chlorophyll=2.0,
        aot_550=0.25,
        wind_speed=10.0,
    ),
]

print("Additional test scenarios defined:")
for i, s in enumerate(test_scenarios, 1):
    print(f"\n{i}. {s}")

print("\n(Run the simulation loop above with each scenario to generate data)")

## 13. Cleanup

Remove temporary files created during simulations.

In [ ]:
# Set to True to clean up temporary OSOAA files
cleanup = False

if cleanup and 'sim' in dir():
    print(f"Cleaning up: {sim.work_dir}")
    sim.cleanup()
    print("Done.")
else:
    if 'sim' in dir():
        print(f"Temporary files preserved in: {sim.work_dir}")
        print(f"Output files: {sim.work_dir / 'Standard_outputs'}")

## Next Steps

With the OSOAA-generated TOA radiances, you can now:

1. **Test atmospheric correction**: Apply the `correct_atmosphere` package to the TOA data
2. **Validate results**: Compare recovered water-leaving radiance with OSOAA's "truth"
3. **Sensitivity studies**: Vary atmospheric/oceanic parameters and assess correction accuracy
4. **Error analysis**: Quantify uncertainties in the atmospheric correction process

### Key Equations

The atmospheric correction removes path radiance to recover water-leaving radiance:

$$L_t = L_{path} + t \cdot L_w$$

Where:
- $L_t$ = TOA radiance (from OSOAA at Level 1)
- $L_{path}$ = Atmospheric path radiance (Rayleigh + aerosol + glint)
- $t$ = Diffuse transmittance
- $L_w$ = Water-leaving radiance (from OSOAA at Level 4)

### References

- Mobley et al. (2016). NASA TM-2016-217551 - Atmospheric Correction Algorithm
- Chami et al. (2015). OSOAA model description, Opt. Express 23, 27829
- Gordon & Wang (1994). SeaWiFS atmospheric correction, Appl. Opt. 33, 443